## Titanic - Machine Learning from Disaster

### The Challenge

The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

### Dataset Description

#### Overview
The data has been split into two groups:

1. **Training set (`train.csv`)**: This set should be used to build your machine learning models. The training set provides the outcome (also known as the "ground truth") for each passenger. Your model will be based on "features" like passengers' gender and class. You can also use feature engineering to create new features.

2. **Test set (`test.csv`)**: This set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include `gender_submission.csv`, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

#### Data Dictionary
| Variable  | Definition                       | Key                                       |
|-----------|----------------------------------|-------------------------------------------|
| survival  | Survival                         | 0 = No, 1 = Yes                           |
| pclass    | Ticket class                     | 1 = 1st, 2 = 2nd, 3 = 3rd                 |
| sex       | Sex                              |                                           |
| Age       | Age in years                     |                                           |
| sibsp     | # of siblings / spouses aboard   |                                           |
| parch     | # of parents / children aboard   |                                           |
| ticket    | Ticket number                    |                                           |
| fare      | Passenger fare                   |                                           |
| cabin     | Cabin number                     |                                           |
| embarked  | Port of Embarkation              | C = Cherbourg, Q = Queenstown, S = Southampton |

##### Variable Notes
- **pclass**: A proxy for socio-economic status (SES)
  - 1st = Upper
  - 2nd = Middle
  - 3rd = Lower
- **age**: Age is fractional if less than 1. If the age is estimated, it is in the form of xx.5
- **sibsp**: The dataset defines family relations in this way:
  - Sibling = brother, sister, stepbrother, stepsister
  - Spouse = husband, wife (mistresses and fiancés were ignored)
- **parch**: The dataset defines family relations in this way:
  - Parent = mother, father
  - Child = daughter, son, stepdaughter, stepson
  - Some children traveled only with a nanny, therefore parch=0 for them.


### Import libraries

In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    auc,
    classification_report,
    confusion_matrix,
    roc_curve,
)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
RANDOM_STATE = 42

### Exploratory Data Analysis

Exploratory Data Analysis (EDA) is a crucial step before diving into machine learning or statistical modeling. It helps us understand the nature of the data, identify patterns, spot anomalies, and form hypotheses.

In [ ]:
# Load train dataframe
train_df = pd.read_csv('data/titanic/train.csv')

In [ ]:
# Take a look at the first few rows
train_df.head(10)

In [ ]:
# Dataframe info
train_df.info()

Let's remove the following columns from the dataset:
- `PassengerId`: Passenger ID, not useful for our analysis
- `Name`: Passenger name, not useful for our analysis
- `Ticket`: Ticket number, not useful for our analysis
- `Cabin`: Cabin number, it has too many missing values
- `Embarked`: Port of Embarkation, not useful for our analysis

In [ ]:
# Let's separate the features from the target
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
target = 'Survived'

# Let's select only the features we want and the target for data analysis
train_df = train_df[features + [target]]

#### 1. Basic Stadistics

In [ ]:
# Generate summary stadistics for numerical columns
train_df.describe()

- **Pclass**: Ranges from 1 to 3. The mean is around 2.31, indicating that there are more passengers in the lower classes.
- **Age**: Ranges from 0.42 to 80 years. The mean age is approximately 29.7 years. Note that the count is less than the total number of passengers, indicating missing values.
- **SibSp**: Ranges from 0 to 8. The mean is approximately 0.52, indicating that most passengers did not have siblings or spouses aboard.
- **Parch**: Ranges from 0 to 6. The mean is around 0.38, suggesting that most passengers were not traveling with parents or children.
- **Fare**: Ranges from 0 to 512.33. The mean fare is approximately 32.2. The standard deviation is high, indicating a wide range of fares.

#### 2. Missing Values

In [ ]:
missing_values = train_df.isnull().sum()
missing_values

We have 177 missing values in the `Age` column.

#### 3. Distribution numerical features

Let's plot histograms for the numerical features.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(20, 10))

colors = {
    'Pclass': ['blue', 'green', 'red'],
    'Sex': ['purple', 'orange'],
    'SibSp': ['gray'] * train_df['SibSp'].nunique(),
    'Parch': ['cyan'] * train_df['Parch'].nunique()
}

# Pclass distribution
train_df['Pclass'].value_counts().sort_index().plot(kind='bar', ax=ax[0, 0], title='Pclass distribution', color=colors['Pclass'], grid=True)
ax[0, 0].grid(axis='x')
# Sex distribution
train_df['Sex'].value_counts().plot(kind='bar', ax=ax[0][1], title="Sex distribution", color=colors['Sex'], grid=True)
ax[0, 1].grid(axis='x')
# Age distribution
train_df['Age'].plot(kind='hist', ax=ax[0][2], bins=30, title="Age distribution", grid=True)
# SibSp distribution
train_df['SibSp'].value_counts().sort_index().plot(kind='bar', ax=ax[1][0], title="SibSp distribution", color=colors['SibSp'], grid=True)
ax[1, 0].grid(axis='x')
# Parch distribution
train_df['Parch'].value_counts().sort_index().plot(kind='bar', ax=ax[1][1], title="Parch distribution", color=colors['Parch'], grid=True)
ax[1, 1].grid(axis='x')
# Fare distribution
train_df['Fare'].plot(kind='hist', ax=ax[1][2], bins=30, title="Fare distribution", grid=True)

plt.tight_layout()
plt.show()

- **Pclass Distribution**: Most passengers are in the 3rd class, followed by 1st and 2nd class.
- **Sex Distribution**: There are more male passengers than female passengers.
- **Age Distribution**: The majority of passengers are in the age range of 20-40 years. The distribution is slightly right-skewed with a smaller number of elderly passengers.
- **SibSp Distribution**: Most passengers traveled without siblings or spouses, with fewer passengers having one or more siblings/spouses.
- **Parch Distribution**: Similarly, most passengers traveled without parents or children aboard.
- **Fare Distribution**: Most fares are on the lower side, with a few passengers paying significantly higher fares. This distribution is right-skewed, indicating potential outliers on the higher side.


Let's create box plots for the numerical features.

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 8))

train_df.boxplot(column="Age", ax=ax[0], boxprops=dict(linestyle='-', linewidth=2, color='red'))
train_df.boxplot(column="SibSp", ax=ax[1], boxprops=dict(linestyle='-', linewidth=2, color='red'))
train_df.boxplot(column="Parch", ax=ax[2], boxprops=dict(linestyle='-', linewidth=2, color='red'))
train_df.boxplot(column="Fare", ax=ax[3], boxprops=dict(linestyle='-', linewidth=2, color='red'))

ax[0].set_title("Boxplot of Age")
ax[1].set_title("Boxplot of SibSp")
ax[2].set_title("Botplot of Parch")
ax[3].set_title("Boxplot of Fare")

for i in range(4):
    ax[i].grid(axis='x')

plt.tight_layout()
plt.show()

- **Boxplot of Age**: The majority of passengers fall within the 20-40 age range, with some outliers on the higher side.
- **Boxplot of SibSp**: Most passengers traveled without siblings or spouses. The higher values (above 2) can be seen as outliers.
- **Boxplot of Parch**: Most passengers traveled without parents or children. The higher values (above 2) can be considered outliers.
- **Boxplot of Fare**: While most fares are on the lower side, there are several outliers on the higher side, indicating passengers who paid significantly more.

#### 4. Distribution target feature

In [ ]:
train_df.dtypes

In [ ]:
warnings.filterwarnings('ignore', category=FutureWarning)

# First at all, we plot target distribution
sns.countplot(x='Survived', data=train_df)
plt.title("Survived distribution")
plt.grid(axis='y')

warnings.resetwarnings()

In [ ]:
warnings.filterwarnings('ignore', category=FutureWarning)

fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(20, 12))

# Convert the Survived column to a string object
train_df['Survived'] = train_df['Survived'].astype(str)

# Now, we plot the distribution of the target by Pclass
sns.countplot(x='Pclass', hue='Survived', data=train_df, ax=axs[0][0]);
axs[0][0].set_title('Survived distribution by Pclass')
axs[0][0].grid(axis='y')

# Plot the distribution of the target by Sex
sns.countplot(x='Sex', hue='Survived', data=train_df, ax=axs[0][1])
axs[0][1].set_title('Survived distribution by Sex')
axs[0][1].grid(axis='y')

# Plot the distribution of the target by Age
sns.boxplot(x='Survived', y='Age', data=train_df, ax=axs[0][2])
axs[0][2].set_title('Survived distribution by Age')

# Plot the distribution of the target by SibSp
sns.countplot(x='SibSp', hue='Survived', data=train_df, ax=axs[1][0])
axs[1][0].set_title('Survived distribution by SibSp')
axs[1][0].grid(axis='y')

# Plot the distribution of the target by Parch
sns.countplot(x='Parch', hue='Survived', data=train_df, ax=axs[1][1])
axs[1][1].set_title('Survived distribution by Parch')
axs[1][1].grid(axis='y')

# Plot the distribution of the target by Fare
sns.boxplot(x='Survived', y='Fare', data=train_df, ax=axs[1][2])
axs[1][2].set_title('Survived distribution by Fare')

plt.show()

warnings.resetwarnings()

From the visualizations, we can observe several patterns, such as:

- A higher proportion of females survived compared to males.
- Higher passenger classes (i.e., 1st class) seem to have a higher survival rate.
- Age, fare, and the number of siblings/spouses (SibSp) and parents/children (Parch) aboard also show variations in the survival rates.

Pairplot:

In [ ]:
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Representa el pairplot
sns.pairplot(train_df, hue = 'Survived', vars = ["Age", "Pclass", "SibSp", "Parch", "Fare"], palette = 'husl', diag_kind = 'kde', kind = 'scatter', height=3);

warnings.resetwarnings()

#### 5. Correlation matrix

In [ ]:
train_df.dtypes

In [ ]:
# Convert the 'Sex' column to numerical values for correlation calculation
corr_df = train_df.copy()
corr_df['Sex'] = corr_df['Sex'].map({'male': 0, 'female': 1})
corr_df['Survived'] = corr_df['Survived'].astype(int)

corr_matrix = corr_df.corr()

print(corr_matrix['Survived'].sort_values(ascending=False))
print(corr_matrix)

sns.heatmap(corr_matrix, cmap='coolwarm', annot=True)
plt.title('Correlation matrix')
plt.show()

There is no strong correlation between the features. 

#### 6. Handling missing values

Let's fill the missing values in the `Age` column with the mean age based on `Sex` and `Pclass`.

In [ ]:
# Group by Sex and Pclass and calculate the mean of Age
mean_ages = train_df.groupby(['Sex', 'Pclass'])['Age'].mean()

# Define function to fill missing values for age
def fill_age(row):
    if pd.isnull(row['Age']):
        return mean_ages[row['Sex'], row['Pclass']]
    else:
        return row['Age']

train_df['Age'] = train_df.apply(fill_age, axis=1)

In [ ]:
# Check if there are missing values
train_df['Age'].isnull().sum()

In [ ]:
train_df.to_csv('data/titanic/train_clean.csv', index=False)

### Data Preprocessing

Let's load the data again:

In [ ]:
titanic_data = pd.read_csv('data/titanic/train_clean.csv')

First at all, le'ts encode the categorical features.

In [ ]:
titanic_data_encoded = pd.get_dummies(titanic_data, columns=["Sex", "Pclass"], drop_first=True)

In [ ]:
titanic_data_encoded.head()

Let's split the data into training and validation sets.

In [ ]:
X = titanic_data_encoded.drop('Survived', axis=1)
y = titanic_data_encoded['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y)

# Let's display the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Let's scale the numerical features.

In [ ]:
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_scaled.head()

### Predictive Modeling

Let's apply different algoritms to predict the survival of passengers.

#### Helper functions

In [ ]:
def perform_search(gs, X_train, y_train):
    """
    Performs a grid search using the provided GridSearchCV o RandomizedSearchCV object on the provided training data.
    
    Parameters:
    gs (GridSearchCV o RandomizedSearchCV): The search object.
    X_train (array-like): The feature matrix for the training data.
    y_train (array-like): The labels for the training data.
    
    Returns:
    The best estimator found during the grid search.
    """
    warnings.filterwarnings('ignore')

    # Fit the GridSearchCV object to the training data
    gs.fit(X_train, y_train)
    
    # Print the best hyperparameters found during the grid search
    print("Best parameters:", gs.best_params_)
    
    # Print the highest mean cross-validated score achieved by the best estimator
    print("Best score:", gs.best_score_)
    
    # Print the best estimator
    print("Best estimator:", gs.best_estimator_)
    
    warnings.resetwarnings()

    # Return the best estimator
    return gs.best_estimator_

In [ ]:
def clf_report(model, X_test, y_test):
    """
    Generates a classification report for the provided model on the provided test data.
    
    Parameters:
    model (sklearn estimator): The model to generate the classification report for.
    X_test (array-like): The feature matrix for the test data.
    y_test (array-like): The labels for the test data.
    """
    # Generate predictions for the test data
    y_pred = model.predict(X_test)
    
    # Print the accuracy score of the model
    print("Accuracy score:", accuracy_score(y_test, y_pred), end="\n\n")

    # Print the classification report
    print(classification_report(y_test, y_pred))
    
    # Print the confusion matrix
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='Blues', fmt='g')

    plt.title('Confusion matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label');

In [ ]:
def plot_ROC_AUC_curves(model, X_train, y_train, X_test, y_test, model_name):
    """
    Plots the ROC (Receiver Operating Characteristic) curves and AUC (Area Under the Curve)
    scores for the given model, using both training and testing data.
    
    Parameters:
    model (object): The classifier model to evaluate.
    X_train (array-like): The feature matrix for the training data.
    y_train (array-like): The labels for the training data.
    X_test (array-like): The feature matrix for the testing data.
    y_test (array-like): The labels for the testing data.
    model_name (str): A name for the model, used in the plot title.
    
    Returns:
    None. Displays the plot using plt.show().
    """
    # Get the probabilities of the positive class for training and testing data
    train_probs = model.predict_proba(X_train)[:, 1]
    test_probs = model.predict_proba(X_test)[:, 1]

    # Calculate the ROC curve for training data
    fpr_train, tpr_train, _ = roc_curve(y_train, train_probs)
    roc_auc_train = auc(fpr_train, tpr_train)
    
    # Calculate the ROC curve for test data
    fpr_test, tpr_test, _ = roc_curve(y_test, test_probs)
    roc_auc_test = auc(fpr_test, tpr_test)

    # Set up the figure and axis for the plot, with a specified size
    plt.figure(figsize=(10, 7))
    
    # Plot the ROC curve for the training data
    plt.plot(fpr_train, tpr_train, color='blue', lw=2, label=f'Train ROC curve (area = {roc_auc_train:.2f})')
    
    # Plot the ROC curve for the testing data
    plt.plot(fpr_test, tpr_test, color='red', lw=2, label=f'Test ROC curve (area = {roc_auc_test:.2f})')
    
    # Plot a gray dashed line representing random guessing
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    
    # Set the limits of the x and y axes
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    
    # Label the axes and the plot
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic (ROC) Curve for {model_name}')
    
    # Add a legend to the plot
    plt.legend(loc='lower right')
    
    # Add a grid to the plot
    plt.grid(True)
    
    # Display the plot
    plt.show()


#### Logistic Regression

We'll perform a grid search to find the best hyperparameters for the model.

In [ ]:
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}

grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    param_grid_lr, 
    cv=5, 
    scoring='accuracy',
    verbose=0, 
    n_jobs=-1,
)

lr = perform_search(grid_search, X_train_scaled, y_train)

Let's evaluate the model on the validation set.

In [ ]:
clf_report(lr, X_test_scaled, y_test)

Now, we plot the ROC curve to determine the performance of the model and overfitting.

In [ ]:
plot_ROC_AUC_curves(lr, X_train_scaled, y_train, X_test_scaled, y_test, 'Logistic Regression')

- The ROC curve for the training data (in blue) has an area under the curve (AUC) of approximately 0.87.

- The ROC curve for the test data (in red) has an AUC of approximately 0.85.

- Both curves are relatively close to each other, which suggests that the model isn't overfitting significantly. If there was a large gap between the training and test ROC curves, it would be more indicative of overfitting.

#### K-Nearest Neighbors (KNN)

Let's perform a Grid Search to find the best hyperparameters for the model.

In [ ]:
param_grid_knn = {
    'n_neighbors': range(1, 30),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
}

grid_search_knn = GridSearchCV(
    KNeighborsClassifier(),
    param_grid_knn, 
    cv=5, 
    scoring='accuracy',
    verbose=0, 
    n_jobs=-1,
)

knn = perform_search(grid_search_knn, X_train_scaled, y_train)

In [ ]:
clf_report(knn, X_test_scaled, y_test)

In [ ]:
plot_ROC_AUC_curves(knn, X_train_scaled, y_train, X_test_scaled, y_test, 'KNN')

There is some overfitting, more than the Logistic Regression model. And the accuracy score is lower.

#### Decision Trees

Let's perform a Grid Search to find the best hyperparameters for the model Decision Tree.

In [ ]:
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 21),
    'min_samples_split': range(2, 21),
    'min_samples_leaf': range(1, 21),
}

grid_search_dt = GridSearchCV(
    DecisionTreeClassifier(random_state=RANDOM_STATE),
    param_grid_dt, 
    cv=5, 
    scoring='accuracy',
    verbose=0, 
    n_jobs=-1,
)

dt = perform_search(grid_search_dt, X_train_scaled, y_train)

In [ ]:
clf_report(dt, X_test_scaled, y_test)

In [ ]:
plot_ROC_AUC_curves(dt, X_train_scaled, y_train, X_test_scaled, y_test, 'Decision Tree')

The result with this model is not improving the accuracy score. And also, there is overfitting.

#### Support Vector Machines (SVM)

Now let's try with SVM.

In [ ]:
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],    
    'gamma': [0.1, 1, 10, 100],
}

random_search_svm = RandomizedSearchCV(
    SVC(random_state=RANDOM_STATE, probability=True),
    param_grid_svm, 
    cv=5, 
    scoring='accuracy',
    verbose=0, 
    n_jobs=-1,
)

svm = perform_search(random_search_svm, X_train_scaled, y_train)

In [ ]:
clf_report(svm, X_test_scaled, y_test)

In [ ]:
plot_ROC_AUC_curves(svm, X_train_scaled, y_train, X_test_scaled, y_test, 'SVM')

We see in the ROC curve that there is overfitting.

#### Random Forest

In [ ]:
param_grid_rf = {
    'n_estimators': [130, 150, 170],
    'criterion': ['entropy'],
    'max_depth': [5],
    'min_samples_split': [2],
    'min_samples_leaf': [2],
}

grid_search_rf = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE),
    param_grid_rf, 
    cv=5, 
    scoring='accuracy',
    verbose=0, 
    n_jobs=-1,
)

rf = perform_search(grid_search_rf, X_train_scaled, y_train)

In [ ]:
clf_report(rf, X_test_scaled, y_test)

In [ ]:
plot_ROC_AUC_curves(rf, X_train_scaled, y_train, X_test_scaled, y_test, 'Random Forest')

There is also overfitting.

#### Gradient Boosting Machines (XGBoost)

Let's try with XGBoost.

In [ ]:
param_grid_xgb = {
    'n_estimators': [150, 200, 250],
    'max_depth': [2, 3, 4, 5,],
    'learning_rate': [0.1, 0.2, 0.3],
}

grid_search_xgb = GridSearchCV(
    xgb.XGBClassifier(random_state=RANDOM_STATE, use_label_encoder=False, eval_metric='logloss'),
    param_grid_xgb, 
    cv=5, 
    scoring='accuracy',
    verbose=0, 
    n_jobs=-1,
)

xgb = perform_search(grid_search_xgb, X_train_scaled, y_train)

In [ ]:
clf_report(xgb, X_test_scaled, y_test)

In [ ]:
plot_ROC_AUC_curves(xgb, X_train_scaled, y_train, X_test_scaled, y_test, 'XGBoost')

#### Neural Networks / Deep Learning

#### AdaBoost

#### Naive Bayes

#### Stacking